---
## Assumptions and rules
---

Use a simple, consistent backtest setup:
- Trade at **next day close** (i.e., use yesterday’s signal as today’s position)
- Ignore transaction costs and slippage (unless you choose to add them)
- Long/cash only for Part A
- Extensions may optionally include shorting, but it is not required

### What “backtest” means here
A backtest simulates how your portfolio would have performed historically if you had followed your trading rules mechanically.


In [ ]:
# Import your functions

---
# Part A: Core Task — Implement One Momentum Strategy
---




---
## Step 1: Libraries 
---

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import quantstats as qs
import warnings

---
## Step 1: Load price data
---


In [ ]:
# Load Data

In [ ]:
# Display the visualisation of the data

---
## Step 2: Backtest framework
---

You will generate a `signal`:
- `1` = long
- `0` = cash

The backtest uses `position = signal.shift(1)` to avoid look-ahead bias.


In [ ]:
# Define the function which will develop

---
## Step 3: Implement one strategy
---

Implement **ONE** of the following by producing a `signal` series (0/1). Comment out the options you are not using.

**Guidance:** Choose which strategy option (A1, A2, or A3) you will implement and use only that signal. For whichever strategy you pick, decide on appropriate parameter values:
- If using **ROC Threshold**: select the lookback window `roc_n` and a percentage threshold. (e.g., `roc_n=10` and `roc_threshold=0.03` means go long if price has risen more than 3% over the last 10 days.)
- If using **MA Crossover**: choose short (`fast_n`) and long (`slow_n`) moving average lengths. (A typical choice might be 20-day vs 50-day MAs, but you can experiment.)
- If using **MACD**: you can use the standard periods (12, 26, 9) for fast EMA, slow EMA, and signal line, or other values if justified.


> **Checkpoint 2:** After developing one trading strategy, considering committing your notebook (e.g., "Completed ROC Threshold").   As with checkpoint 1, note the preference for more frequent commits together with branches and pull requests as part of the collaboration.   This comment applies to all checkpoints.


---
### Option A1: ROC Threshold
---

**Rule:** Buy if the price increased by more than a threshold over the last `roc_n` days.


In [ ]:
# Develop the ROC indicator

---
### Option A2: Moving Average Crossover
---

**Rule:** Buy when a fast moving average is above a slow moving average.


In [ ]:
# Develop the Moving Average Crossover indicator

---
### Option A3: MACD Signal
---

**Rule:** Buy when MACD is above its signal line.
- `MACD = EMA(12) - EMA(26)`
- `Signal line = EMA(9) of MACD`


In [ ]:
# Develop the MACD indicator

---
## Step 4: Backtest and report (Part A)
---

Run your backtest and report:
- Total return
- Number of trades
- Equity curve
- Price chart with buy/sell markers


In [ ]:
# Run the backtest amd report the results

---
## Part A short write-up
---

**Write 2–5 sentences:**
- What does your indicator measure?
- When does your strategy enter/exit?
- Why might this work (behavioural explanation)?

**Answer:**
- TODO

> **Checkpoint 3:** After completing the Part A strategy implementation and backtest, consider committing your notebook (e.g., "Finished Part A strategy").


---
# Part B: Extensions — Multiple Strategies and Composite Score
---

This section extends Part A by implementing multiple strategies and combining them into a composite score.


---
## Extension B1: Implement two additional strategies
---

Implement the **two strategies you did not choose** in Part A.

Use the provided functions to create signals for each new strategy. For example, if you chose ROC in Part A, you can use `signal_ma_crossover(...)` for one of the extensions and `signal_macd(...)` for the other. Ensure each strategy's signal is a 0/1 series like before, then backtest them just as in Part A.

### Deliverables
- Backtest each strategy under the same assumptions
- Produce a simple comparison table: total return, number of trades

> **Checkpoint 4:** Once you have completed a strategy, commit your progress (e.g., "Completed MA Crossover").


In [ ]:
# Complete the other strategy 1

In [ ]:
# Complete the other strategy 2

---
## Extension B2: Composite momentum score strategy
---

Build a composite score from ROC, MA crossover, and MACD.

### Requirements
- Convert each indicator into a **standardised sub-score** (e.g., -2..+2)
- Choose weights that sum to 1:
  $w_{roc} + w_{ma} + w_{macd} = 1$
- Composite score:
  $Score_t = w_{roc}s^{roc}_t + w_{ma}s^{ma}_t + w_{macd}s^{macd}_t$
- Trading rule (long/cash):
  - Long if `Score >= x`
  - Cash if `Score < x`

**Guidance:** First, convert each strategy's signal into a numerical sub-score on a common scale (we use -2 to +2 for ROC buckets, and -1/+1 for MA and MACD signals as given above). Next, assign weights to each component based on how much emphasis you want to give it. (If unsure, equal weights is a reasonable starting point.) Make sure the weights sum to 1. Then choose a threshold `x` for the composite score that determines when to go long. For example, `x = 0` would mean any positive composite signal triggers a long position, whereas a higher threshold (like 0.5) would require stronger agreement among indicators to enter a trade.


In [ ]:
# Complete the function which trades based on a composite score

---
## Extension B2 short write-up
---

**Write 4–8 sentences:**
- Why did you choose your weights and threshold `x`?
- Did the composite score outperform your best single strategy?
- What is one limitation of this backtest?

**Answer:**
- TODO

> **Checkpoint 5:** You have now completed the composite strategy implementation. Review your results and then make a final commit of the notebook (e.g., "Completed momentum strategies assignment").
